In [10]:
import os
import math
import pandas as pd

In [11]:
data_path = os.getcwd() + "/dataset/original_data"
first_processing_path = os.getcwd() +"/dataset/raw_labelled"
second_processing_path = os.getcwd() +"/dataset/windows"
final_dataset_path = os.getcwd() +"/dataset"

window_length = 4*64

In [12]:
'''
people[] -> 存所有在資料夾中的txt檔

'''
people = [] #S01R01.txt~S10R01.txt

for person in os.listdir(data_path):
    if '.txt' in person:
        people.append(person)


del person

In [13]:
def label_prefog(dataset, window_length):
# def label_prefog(dataset, window_length = 1):
    # Drop all the rows for which Action is 0 or the rows which are not part of the experiment
    # 將所有的Action==0(未實驗)的部分刪除，並不回傳任何值(implace的部分)
    dataset.drop(index = list(dataset[dataset['Action'] == 0].index),
                 inplace = True)
    
    # window_length = int(64*window_length)
    #print(window_length)
    fog_index = []
    for i in dataset.index:
        if dataset.loc[i, 'Action'] == 2: # 取index和'Action'對應到的資料
            fog_index.append(i) # 將發生fog的index存入fog_index這個array
    fog_index
    
    # print(fog_index)
    
    start_indices = []
    for i in fog_index:
        if (dataset.loc[i-1, 'Action'] != dataset.loc[i, 'Action']):
            start_indices.append(i) # 將發生fog之前的index存入start_indices這個array
    
    # print(start_indices)

    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-window_length, start)] # prefog_start把FOG之前的windows標記起來
        prefog.append(prefog_start)
        
    # print(prefog) # 2D array

    prefog = [item for sublist in prefog for item in sublist] # 把[][]轉換成[]

    # print(prefog) # 1D array

    for i in prefog:
        dataset.loc[i,'Action'] = 3 # 把紀錄在Pre-fog的index action 記錄成3
    dataset['Action'] = dataset['Action'] - 1 # 所有action編號-1 使0=walk 1=fog 2= pre-fog
    return dataset

In [14]:
dataset = pd.DataFrame()
last_name = '' # 紀錄哪個是最終的病患
patients = [] # 病患ID list

for person in people:
    name = person.split('R')[0]

    if name != last_name:
        dataset = dataset.drop(index=dataset.index)
        patients.append(name)
    last_name = name
    print(name)

    file = data_path + "\\" + person

    if name in person:
        temp = pd.read_csv(file, delimiter = " ", header = None)
        print(person, ' is read', end = "\t")

        if 2 in temp[max(temp.columns)].unique():
            print("Adding {} to dataset".format(person), end = "\t")
            temp.columns = [
                                'time',
                                'A_F', 'A_V', 'A_L',
                                'L_F', 'L_V', 'L_L',
                                'T_F', 'T_V', 'T_L',
                                'Action'                           
                                ]
            
            temp = label_prefog(temp, window_length).reset_index(drop = True)
            temp['name'] = name
            print("{} is labelled".format(person))
            dataset = pd.concat([dataset, temp],axis = 0)
        print('')
    
        dataset.reset_index(drop = True, inplace = True)
        
        to_name = first_processing_path + "/win_" + name + ".csv"
    # 存入/raw_labelled中
    
    if not dataset.empty:
        dataset.to_csv(to_name, index = False)

del last_name, name, person, people, dataset, temp, to_name


S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	
S04
S04R01.txt  is read	
S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled

S05
S05R02.txt  is read	Adding S05R02.txt to dataset	S05R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	
S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled

S07
S07R02.txt  is read	Adding S07R02.txt to dataset	S07R02.txt is labelled

S08
S08R01.txt  is read	Adding S08R01.txt to dataset	S08R01.txt is labelled

S

In [15]:
def create_window(act,window_length,dataframe):

  indices = list(dataframe[dataframe.Action == act].index)
  time = []
  time_count = 0
  for j in indices:
    time.append(dataframe.loc[j, 'time'])


  # indices記錄所有Action==act的index
  groups = [] # 用來暫存一組(同action)資料的, 型態是[][]
  temp = [] # 用來暫存一行資料的
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break # 如果i已經來到最後的話就break
    temp.append(indices[i])
    #time_count = time_count + 1
    if time[i+1]-16 > time[i]: #如果下個index不是連續的話, 就將前面這些存成第一組
      group_count+=1
      #time_count = time_count + 1
      groups.append(temp)
      temp = []

  #print(groups)

  fs = 64
  # window_length = 1
  # window_length = int(window_length*fs)

  final_dataframe = pd.DataFrame()
  sumOfAct=0

  for i in groups: # group[][]的每一行i
    required = math.floor(len(i)/int(window_length))

    
    sumOfAct= sumOfAct+required

    req_index = i[0:(required*int(window_length))]

    #print(req_index)
    # concat([要結合的data集合], axis=0是方向為直的)
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  
  

  return final_dataframe

In [16]:
for patient in patients:

  name = first_processing_path + "/win_"+ patient +".csv"

  if os.path.isfile(name):
    dataset = pd.read_csv(name)

    activities = []
    # print(patient)
    for act in range(3):
      activities.append(create_window(act,window_length,dataset))
    to_write = pd.concat(activities,axis = 0)
    to_path = second_processing_path + "/windowed_" + patient + ".csv"
    to_write.to_csv(to_path,index = False)

del name, to_path, to_write, dataset, activities, patient, act

In [17]:
total_dataset=pd.DataFrame()
for patient in patients:
  name = second_processing_path + "\\windowed_" + patient + ".csv"

  if os.path.isfile(name):
    dataset = pd.read_csv(name)
    total_dataset = total_dataset.append(dataset)

to_path = final_dataset_path + "/total.csv"
total_dataset.to_csv(to_path,index = False)

del name, dataset, to_path, patient

In [18]:
data_path = os.getcwd() + "/dataset/total.csv"
dataset = pd.read_csv(data_path)

import math

activities = []
for act in range(3):
    activities.append(create_window(act,window_length,dataset))
to_write = pd.concat(activities, axis = 0)
to_path = os.getcwd() + "/dataset/total_concentrated.csv"
to_write.to_csv(to_path)